In [4]:
from PIL import Image
import numpy as np
import skimage
import skimage.morphology
import skimage.feature
import skan
from scipy import ndimage as ndi
from tifffile import imsave
from scipy.interpolate import interp1d



In [13]:
skeleton_node = np.zeros([3,np.shape(skeleton_image)[0],np.shape(skeleton_image)[1],np.shape(skeleton_image)[2]])
skeleton_node[0,:,:,:] = skeleton_image

summary=skan.summarize(skeleton_obj)
net = np.zeros([5,len(summary.index)])
net[0,:]=summary['skeleton-id']
root=[]
for i in summary.index:
    if(summary['branch-type'][i]==1):
            root.append(i)
    for j in summary.index[i+1:]:
        if(summary['node-id-dst'][i]==summary['node-id-src'][j]):
            if(net[1,i]==0):
                net[1,i] = summary['skeleton-id'][j]
            elif(net[2,i]==0):
                net[2,i] = summary['skeleton-id'][j]
            else:
                skeleton_node[1,int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])] = 1
                print("warning, more than 2 children attached to branch ",i)
            if(net[3,j]==0):
                net[3,j] = summary['skeleton-id'][i]
            elif(net[4,j]==0):
                net[4,j] = summary['skeleton-id'][i]
            else:
                skeleton_node[1,int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])] = 1
                print("warning, more than 2 parents attached to branch ",j)


warning, more than 2 children attached to branch  305
warning, more than 2 parents attached to branch  846
warning, more than 2 parents attached to branch  874
warning, more than 2 children attached to branch  1212


In [14]:
print(np.shape(skeleton_node))
imsave('images/skeleton.tif',np.einsum('ijk->kij', ndi.binary_dilation(skeleton_node[0,:,:,:])))
skeleton_node[2,:,:,:]=np.load('np arrays/foreground.npy')
imsave('images/skeleton_node.tif',np.einsum('ijkl->lijk', skeleton_node))

(3, 512, 512, 271)


In [ ]:
soma_mask = ndi.binary_dilation(np.load('np arrays/soma_mask.npy'))
roots=[i for i in summary.index if (summary['branch-type'][i] == 1 and
			( soma_mask[int(summary['image-coord-dst-0'][i]),int(summary['image-coord-dst-1'][i]),int(summary['image-coord-dst-2'][i])]==1 or
            soma_mask[int(summary['image-coord-src-0'][i]),int(summary['image-coord-src-1'][i]),int(summary['image-coord-src-2'][i])]==1 )
            )]
